In [33]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


'''
#### File Tree ####
다음과 같이 구성하고 실행하세요

- source
    - test.ipynb
- train
  - 1
    - 1_01.jpg
    - ...
  - 2
    - 2_02.jpg
    - ...
- validation
  - 1
    - 1_01.jpg
    - ...
  - 2
    - 2_02.jpg
    - ...

'''

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../train'
validation_data_dir = '../validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50 ## 임의의 훈련 진행횟수
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#### 순차적인(일렬) 모델 생성
## 히든레이어 1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## 히든레이어 2 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## 히든레이어 3
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## Drop 레이어
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(20))
model.add(Activation('softmax'))

## 모델 컴파일
## 2개 (ex:개 or 고양이 분류)) 일땐 binary 를 사용하며 그이상일 경우 categorical 사용
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
## ImageDataGenerator 는 임의의 이미지를 생성하는 함수이며
## 여기선 훈련용 데이터가 부족하기 때문에 원본 사진에서 스케일링, 사진회전, 좌우반전등을 통해 훈련용 이미지를 증가시킨다
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
## 훈련용 이미지를 지외하고 검증용, 테스트용은 원본사진으로 하기위해 리스케일만 진행한다.
test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

## flow_from_directory 는 앞서 생성한 임의의 이미지를 불러오는 함수
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

## 모델 훈련
## Tensor flow GPU 모드를 사용하면 약 90% 빨라진다.
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

## 모델 학습이 오래걸리기 때문에 저장하여 나중에 다양하게 활용한다.
## ex) 보강학습, 모델검증 등등
model.save_weights('first_try.h5')

Found 491 images belonging to 20 classes.
Found 226 images belonging to 20 classes.
Found 226 images belonging to 20 classes.
Epoch 1/50
125/125 [==============================] - 46s 365ms/step - loss: 3.0594 - accuracy: 0.0596 - val_loss: 2.9955 - val_accuracy: 0.0778
Epoch 2/50
125/125 [==============================] - 45s 358ms/step - loss: 2.8287 - accuracy: 0.1338 - val_loss: 2.8004 - val_accuracy: 0.2665
Epoch 3/50
125/125 [==============================] - 45s 359ms/step - loss: 2.4990 - accuracy: 0.2399 - val_loss: 1.1944 - val_accuracy: 0.3925
Epoch 4/50
125/125 [==============================] - 45s 359ms/step - loss: 2.2292 - accuracy: 0.3237 - val_loss: 2.2450 - val_accuracy: 0.3984
Epoch 5/50
125/125 [==============================] - 45s 362ms/step - loss: 1.9663 - accuracy: 0.3949 - val_loss: 2.2661 - val_accuracy: 0.4485
Epoch 6/50
125/125 [==============================] - 45s 359ms/step - loss: 1.8168 - accuracy: 0.4317 - val_loss: 1.3065 - val_accuracy: 0.4785
Epoc